In [48]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [49]:
import librosa
from sklearn.preprocessing import StandardScaler, LabelEncoder
from joblib import Parallel, delayed
import scipy.signal as signal
from tqdm import tqdm
import gc

In [50]:
# Reading the CSV files into DataFrames
metadata_df = pd.read_csv('/kaggle/input/birdclef-2022/train_metadata.csv')
taxonomy_df = pd.read_csv('/kaggle/input/birdclef-2022/eBird_Taxonomy_v2021.csv')

#filtering the metadata to havings ratings of >= 3
filtered_metadata_df = metadata_df[metadata_df['rating'] >= 3]
print(f"\nTotal number of after filtering ratings of >= 3:")
len(filtered_metadata_df)


Total number of after filtering ratings of >= 3:


12646

In [51]:
from collections import Counter
import re

# List of columns to analyze
columns_to_analyze = ['secondary_labels', 'type']

# Function to analyze each column with individual string components, case-insensitive and space-insensitive
def analyze_column_individual_case_insensitive(df, column_name):
    print(f"\nAnalyzing individual values within the column (case-insensitive and space-insensitive): {column_name}")
    
    # Initialize a Counter to count each unique item across all entries
    item_counter = Counter()
    
    # Iterate over each entry in the column
    for entry in df[column_name].dropna():
        # Convert the string to a list, make each item lowercase, and remove spaces
        items = [re.sub(r'\s+', '', item.lower()) for item in eval(entry)]  # Lowercase and remove spaces
        # Update the counter with items in the list
        item_counter.update(items)
    
    # Print total unique items and counts for each
    #unique_items = len(item_counter)
    #print(f"Total number of unique items: {unique_items}")
    #print(f"\nCount of occurrences for each unique item (case-insensitive and space-insensitive):\n{item_counter}")
    
    return item_counter
    
# Analyze the 'type' column
type_counts = analyze_column_individual_case_insensitive(filtered_metadata_df, 'type')

noisy_terms = ['wing', 'wings', 'water', 'splash', 'rain', 'ground', 'background', 'noise', 'anthropogenic', 'traffic', 'street']

# Filter types that contain any of the noisy terms
noisy_types = [item for item in type_counts if any(term in item for term in noisy_terms)]

print("\nTypes identified as containing noise (including specified terms):")
print(noisy_types)

# Function to filter DataFrame by removing rows where all 'type' values match noisy types
def filter_noisy_rows(df, noisy_types):
    def has_only_noisy_types(type_list):
        # Clean and normalize type_list
        items = [re.sub(r'\s+', '', item.lower()) for item in eval(type_list)]
        # Check if all items are in noisy_types
        return all(item in noisy_types for item in items)
    
    # Filter the DataFrame
    filtered_df = df[~df['type'].apply(has_only_noisy_types)]
    
    return filtered_df

# Apply the filter to remove rows with only noisy types
filtered_metadata_df_cleaned = filter_noisy_rows(filtered_metadata_df, noisy_types)

# Display the total number of filtered samples
print(f"\nTotal number of samples without noisy types:")
len(filtered_metadata_df_cleaned)



Analyzing individual values within the column (case-insensitive and space-insensitive): type

Types identified as containing noise (including specified terms):
['traffic', 'wingflutters', 'wingsounds', 'wingsbeats', 'insectsoundsinbackground', 'ducksflushingoffwater', 'anthropogenic', 'rain', 'wings', 'trafficonfreeway', 'anthropogenic:', 'wingbeats', 'wingflapping', 'wingnoise', 'wingsinflight', 'wingsoundsascoveyflushed', 'wingwhir', 'wingflaps', 'wingbeatasflyingover', 'splash', 'waterdroplet', 'callsonwater', 'communicatingwhilefeedingontheground', 'anthropogenicbackgroundnoise', 'whistlesofduckwingsinflight', 'callfromthewater', 'noiseofwings', 'wingsplashes', 'noiseofnestingcolony', 'streettraffic', 'callsfromground', 'watersplashing', 'alarmfromwater', 'inclwingbeats', 'wingbeat', 'somdobaterdasasas.-soundofbeatingofwings.', 'landingonwater', 'birdisflyingoutofthewater(wingssound)', 'audiblewingbeats', 'flapwings', 'greylaggoosewingflapping', 'landingonthewater', 'wingwhirr', '

12608

In [52]:
# Merge with taxonomy on scientific name
merged_df = pd.merge(filtered_metadata_df_cleaned, taxonomy_df, left_on='scientific_name', right_on='SCI_NAME', how='left')

# Drop specified columns after merging
merged_df = merged_df.drop(columns=['primary_label', 'PRIMARY_COM_NAME', 'secondary_labels', 'author', 'license', 'rating', 'REPORT_AS', 'SCI_NAME', 'time', 'url', 'SPECIES_GROUP'])

# Display DataFrames after filtering and merging.
print("DataFrame after filtering and merging:")
merged_df.head()

DataFrame after filtering and merging:


,type,latitude,longitude,scientific_name,common_name,filename,TAXON_ORDER,CATEGORY,SPECIES_CODE,ORDER1,FAMILY
0,['call'],19.8801,-155.7254,Euodice cantans,African Silverbill,afrsil1/XC175522.ogg,30031,species,afrsil1,Passeriformes,Estrildidae (Waxbills and Allies)
1,"['call', 'song']",16.2901,-16.0321,Euodice cantans,African Silverbill,afrsil1/XC177993.ogg,30031,species,afrsil1,Passeriformes,Estrildidae (Waxbills and Allies)
2,"['alarm call', 'call']",17.0922,54.2958,Euodice cantans,African Silverbill,afrsil1/XC205893.ogg,30031,species,afrsil1,Passeriformes,Estrildidae (Waxbills and Allies)
3,['flight call'],21.4581,-157.7252,Euodice cantans,African Silverbill,afrsil1/XC207431.ogg,30031,species,afrsil1,Passeriformes,Estrildidae (Waxbills and Allies)
4,['flight call'],21.4581,-157.7252,Euodice cantans,African Silverbill,afrsil1/XC207432.ogg,30031,species,afrsil1,Passeriformes,Estrildidae (Waxbills and Allies)


In [53]:
final_df = merged_df.drop(columns=['type', 'latitude', 'longitude', 'scientific_name', 'TAXON_ORDER', 'CATEGORY', 'SPECIES_CODE', 'FAMILY'])  

final_df.to_csv('final_df.csv', index=False)

final_df.head()

,common_name,filename,ORDER1
0,African Silverbill,afrsil1/XC175522.ogg,Passeriformes
1,African Silverbill,afrsil1/XC177993.ogg,Passeriformes
2,African Silverbill,afrsil1/XC205893.ogg,Passeriformes
3,African Silverbill,afrsil1/XC207431.ogg,Passeriformes
4,African Silverbill,afrsil1/XC207432.ogg,Passeriformes


In [55]:
# Define the base path for audio files
base_path = '/kaggle/input/birdclef-2022/train_audio'

delta_width = 3
# Bandpass filter function for typical birdsong frequencies (1-10 kHz)
def bandpass_filter(y, sr, lowcut=1000, highcut=10000):
    sos = signal.butter(10, [lowcut, highcut], btype='band', fs=sr, output='sos')
    filtered = signal.sosfilt(sos, y)
    return filtered

# Function to extract audio features
def extract_features(y, sr):
    features = {}
    
    # Apply bandpass filter
    y = bandpass_filter(y, sr)
    
    return features

In [56]:
# Function to process data in batches
def process_in_batches(df, batch_size=100):
    all_features = []
    num_batches = len(df) // batch_size + (1 if len(df) % batch_size != 0 else 0)
    
    for batch_num in tqdm(range(num_batches), desc="Processing Batches"):
        batch_df = df.iloc[batch_num * batch_size : (batch_num + 1) * batch_size]
        
        batch_features = []
        for filename in batch_df['filename']:
            file_path = os.path.join(base_path, filename)
            
            y, sr = librosa.load(file_path, sr=44100)  # Set sample rate to 44100 Hz
            features = extract_features(y, sr)
            batch_features.append(features)
        
        # Convert batch features to DataFrame
        features_df = pd.DataFrame(batch_features)
        
        # Concatenate features with the original batch DataFrame
        batch_combined = pd.concat([batch_df.reset_index(drop=True), features_df], axis=1)
        all_features.append(batch_combined)
        
        # Clear memory
        del batch_features, y
        gc.collect()

    # Concatenate all batches to form the final DataFrame
    final_data = pd.concat(all_features, ignore_index=True)
    return final_data

# Apply batch processing to the DataFrame
extra_data = process_in_batches(final_df, batch_size=100)

# Save the final DataFrame to a CSV file
extra_data.to_csv("extra_data.csv", index=False)

# Display DataFrames after feature extraction
print("DataFrame after feature extraction:")


  0%|          | 7/12608 [04:17<128:37:44, 36.75s/it]?it/s]

Processing Batches: 100%|██████████| 127/127 [47:23<00:00, 22.39s/it]

DataFrame after feature extraction:


In [57]:
extra_data.head()

,common_name,filename,ORDER1
0,African Silverbill,afrsil1/XC175522.ogg,Passeriformes
1,African Silverbill,afrsil1/XC177993.ogg,Passeriformes
2,African Silverbill,afrsil1/XC205893.ogg,Passeriformes
3,African Silverbill,afrsil1/XC207431.ogg,Passeriformes
4,African Silverbill,afrsil1/XC207432.ogg,Passeriformes


In [58]:
from sklearn.model_selection import train_test_split

In [65]:
!pip install torchlibrosa

In [66]:
import cv2
import audioread
import logging
import gc
import os
import sys
import random
import time
import warnings

import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata

from contextlib import contextmanager
from joblib import Parallel, delayed
from pathlib import Path
from typing import Optional
from sklearn.model_selection import StratifiedKFold, GroupKFold

from albumentations.core.transforms_interface import ImageOnlyTransform
from torchlibrosa.stft import LogmelFilterBank, Spectrogram
from torchlibrosa.augmentation import SpecAugmentation
from tqdm import tqdm

import albumentations as A
import albumentations.pytorch.transforms as T

import matplotlib.pyplot as plt

In [67]:
SEED = 42
DATA_PATH = "/kaggle/input/birdclef-2022"
AUDIO_PATH = '/kaggle/input/birdclef-2022/train_audio'
MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])
NUM_WORKERS = 4
CLASSES = sorted(os.listdir(AUDIO_PATH))
NUM_CLASSES = len(CLASSES)
class AudioParams:
    sr = 32000
    duration = 5
    # Melspectrogram
    n_mels = 224
    fmin = 20
    fmax = 16000

Sf_data = extra_data
#paths = extra_data["filename"].values

Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
for n, (trn_index, val_index) in enumerate(Fold.split(Sf_data, Sf_data['ORDER1'])):
    Sf_data.loc[val_index, 'kfold'] = int(n)
Sf_data['kfold'] = Sf_data['kfold'].astype(int)

Sf_data.to_csv('Sf_data.csv', index=False)

print(Sf_data.shape)
Sf_data.head()

(12608, 4)


,common_name,filename,ORDER1,kfold
0,African Silverbill,afrsil1/XC175522.ogg,Passeriformes,0
1,African Silverbill,afrsil1/XC177993.ogg,Passeriformes,3
2,African Silverbill,afrsil1/XC205893.ogg,Passeriformes,1
3,African Silverbill,afrsil1/XC207431.ogg,Passeriformes,2
4,African Silverbill,afrsil1/XC207432.ogg,Passeriformes,3


In [68]:
def compute_melspec(y, params):
    melspec = librosa.feature.melspectrogram(
        y=y, sr=params.sr, n_mels=params.n_mels, fmin=params.fmin, fmax=params.fmax,
    )

    melspec = librosa.power_to_db(melspec).astype(np.float32)
    return melspec


def mono_to_color(X, eps=1e-6, mean=None, std=None):
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)

    # Normalize to [0, 255]
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V

In [83]:
def Audio_to_Image(path, params):
    y, sr = sf.read(path, always_2d=True)
    y = np.mean(y, 1) # there is (X, 2) array
    #y = crop_or_pad(y, params.duration * params.sr, sr=params.sr, train=True, probs=None)
    image = compute_melspec(y, params)
    image = mono_to_color(image)
    image = image.astype(np.uint8)
    return image

def save_(path, AudioParams):
    # Extract the class and file name from the source path
    # Example: '/kaggle/input/birdclef-2022/train_audio/afrsil1/XYZ123.ogg'
    class_name = path.split('/')[-2]  # 'afrsil1'
    audio_file = path.split('/')[-1]  # 'XYZ123.ogg'

    # Create the target directory inside '/kaggle/working/'
    save_dir = os.path.join("/kaggle/working", class_name)
    os.makedirs(save_dir, exist_ok=True)  # Ensure target directory exists
    
    # Define the save path for the transformed audio data
    save_path = os.path.join(save_dir, f"{audio_file}.npy")  # Save as .npy file
    
    # Transform the audio to an image or other format and save it
    audio_image = Audio_to_Image(path, AudioParams)
    
    # Save the transformed data as a .npy file
    np.save(save_path, audio_image)
    print(f"Saved {save_path}")

In [84]:
from joblib import Parallel, delayed

NUM_WORKERS = 4

# Assuming paths is a list of audio file paths you want to process
paths = [os.path.join("/kaggle/input/birdclef-2022/train_audio", cls, f) for cls in CLASSES for f in os.listdir(os.path.join("/kaggle/input/birdclef-2022/train_audio", cls))]

# Apply the save function to each audio file in parallel
_ = Parallel(n_jobs=NUM_WORKERS)(
    delayed(save_)(audio_path, AudioParams) for audio_path in tqdm(paths)
)








  0%|          | 0/14852 [00:00<?, ?it/s]





  0%|          | 4/14852 [00:00<06:53, 35.95it/s]





  0%|          | 8/14852 [00:05<3:03:45,  1.35it/s]





  0%|          | 12/14852 [00:05<1:52:37,  2.20it/s]





  0%|          | 16/14852 [00:05<1:17:09,  3.20it/s]





  0%|          | 20/14852 [00:06<59:39,  4.14it/s]  





  0%|          | 24/14852 [00:06<45:58,  5.38it/s]





  0%|          | 28/14852 [00:07<37:40,  6.56it/s]





  0%|          | 32/14852 [00:08<50:31,  4.89it/s]





  0%|          | 36/14852 [00:08<47:03,  5.25it/s]





  0%|          | 40/14852 [00:10<56:29,  4.37it/s]





  0%|          | 44/14852 [00:10<54:20,  4.54it/s]





  0%|          | 48/14852 [00:11<56:42,  4.35it/s]





  0%|          | 52/14852 [00:13<1:00:35,  4.07it/s]





  0%|          | 56/14852 [00:13<46:43,  5.28it/s]  





  0%|          | 60/14852 [00:13<37:46,  6.53it/s]





  0%|          | 64/14852 [00:14<33:14,  7.41it/s]





  0%|          | 68/14852 [00:14<28:22,  

Saved /kaggle/working/afrsil1/XC207432.ogg.npy
Saved /kaggle/working/afrsil1/XC125458.ogg.npy
Saved /kaggle/working/afrsil1/XC445331.ogg.npy
Saved /kaggle/working/afrsil1/XC371226.ogg.npy
Saved /kaggle/working/afrsil1/XC317039.ogg.npy
Saved /kaggle/working/afrsil1/XC234994.ogg.npy
Saved /kaggle/working/akekee/XC210201.ogg.npy
Saved /kaggle/working/akepa1/XC27348.ogg.npy
Saved /kaggle/working/akepa1/XC175513.ogg.npy
Saved /kaggle/working/akepa1/XC147002.ogg.npy
Saved /kaggle/working/amewig/XC301850.ogg.npy
Saved /kaggle/working/amewig/XC293789.ogg.npy
Saved /kaggle/working/amewig/XC396690.ogg.npy
Saved /kaggle/working/amewig/XC593011.ogg.npy
Saved /kaggle/working/amewig/XC522375.ogg.npy
Saved /kaggle/working/amewig/XC187408.ogg.npy
Saved /kaggle/working/amewig/XC531510.ogg.npy
Saved /kaggle/working/amewig/XC187411.ogg.npy
Saved /kaggle/working/amewig/XC305022.ogg.npy
Saved /kaggle/working/amewig/XC519619.ogg.npy
Saved /kaggle/working/amewig/XC335125.ogg.npy
Saved /kaggle/working/amewig/







  5%|▍         | 680/14852 [01:37<33:07,  7.13it/s]





  5%|▍         | 684/14852 [01:38<48:05,  4.91it/s]





  5%|▍         | 688/14852 [01:39<45:05,  5.24it/s]





  5%|▍         | 692/14852 [01:40<49:44,  4.74it/s]





  5%|▍         | 696/14852 [01:42<1:01:06,  3.86it/s]





  5%|▍         | 700/14852 [01:43<1:07:24,  3.50it/s]

Saved /kaggle/working/afrsil1/XC205893.ogg.npy
Saved /kaggle/working/afrsil1/XC209513.ogg.npy
Saved /kaggle/working/akekee/XC174954.ogg.npy
Saved /kaggle/working/akekee/XC27306.ogg.npy
Saved /kaggle/working/akepa1/XC175511.ogg.npy
Saved /kaggle/working/akiapo/XC122693.ogg.npy
Saved /kaggle/working/akiapo/XC124705.ogg.npy
Saved /kaggle/working/amewig/XC403005.ogg.npy
Saved /kaggle/working/amewig/XC305020.ogg.npy
Saved /kaggle/working/amewig/XC513990.ogg.npy
Saved /kaggle/working/amewig/XC152873.ogg.npy
Saved /kaggle/working/amewig/XC172889.ogg.npy
Saved /kaggle/working/amewig/XC531503.ogg.npy
Saved /kaggle/working/amewig/XC509723.ogg.npy
Saved /kaggle/working/amewig/XC150063.ogg.npy
Saved /kaggle/working/amewig/XC513991.ogg.npy
Saved /kaggle/working/amewig/XC524663.ogg.npy
Saved /kaggle/working/amewig/XC150015.ogg.npy
Saved /kaggle/working/amewig/XC509697.ogg.npy
Saved /kaggle/working/amewig/XC519789.ogg.npy
Saved /kaggle/working/amewig/XC298401.ogg.npy
Saved /kaggle/working/amewig/XC17







  5%|▍         | 704/14852 [01:45<1:21:36,  2.89it/s]





  5%|▍         | 708/14852 [01:46<1:19:35,  2.96it/s]





  5%|▍         | 712/14852 [01:47<1:06:08,  3.56it/s]





  5%|▍         | 716/14852 [01:47<56:40,  4.16it/s]  





  5%|▍         | 720/14852 [01:48<49:24,  4.77it/s]





  5%|▍         | 724/14852 [01:48<41:15,  5.71it/s]





  5%|▍         | 728/14852 [01:49<32:44,  7.19it/s]

...







 68%|██████▊   | 10036/14852 [24:54<06:22, 12.58it/s]





 68%|██████▊   | 10040/14852 [24:55<09:32,  8.41it/s]





 68%|██████▊   | 10044/14852 [24:55<08:47,  9.11it/s]





 68%|██████▊   | 10048/14852 [24:55<08:30,  9.40it/s]





 68%|██████▊   | 10052/14852 [24:56<08:03,  9.92it/s]





 68%|██████▊   | 10056/14852 [24:56<07:32, 10.61it/s]





 68%|██████▊   | 10060/14852 [24:57<08:42,  9.17it/s]





 68%|██████▊   | 10064/14852 [24:57<09:00,  8.86it/s]





 68%|██████▊   | 10068/14852 [24:57<08:51,  9.00it/s]





 68%|██████▊   | 10072/14852 [24:58<06:51, 11.62it/s]





 68%|██████▊   | 10076/14852 [24:58<05:31, 14.39it/s]





 68%|██████▊   | 10080/14852 [24:58<05:14, 15.15it/s]





 68%|██████▊   | 10084/14852 [24:58<04:32, 17.52it/s]





 68%|██████▊   | 10088/14852 [24:58<04:11, 18.92it/s]





 68%|██████▊   | 10092/14852 [24:58<04:11, 18.94it/s]





 68%|██████▊   | 10096/14852 [24:59<04:32, 17.42it/s]





 68%|██████▊   | 10100/14852 [24:59<05:12, 15.23it

Saved /kaggle/working/normoc/XC131455.ogg.npy
Saved /kaggle/working/normoc/XC321865.ogg.npy
Saved /kaggle/working/normoc/XC357935.ogg.npy
Saved /kaggle/working/normoc/XC608254.ogg.npy
Saved /kaggle/working/normoc/XC541496.ogg.npy
Saved /kaggle/working/normoc/XC503811.ogg.npy
Saved /kaggle/working/normoc/XC662554.ogg.npy
Saved /kaggle/working/normoc/XC184932.ogg.npy
Saved /kaggle/working/normoc/XC131895.ogg.npy
Saved /kaggle/working/normoc/XC320715.ogg.npy
Saved /kaggle/working/normoc/XC661656.ogg.npy
Saved /kaggle/working/normoc/XC130971.ogg.npy
Saved /kaggle/working/normoc/XC570236.ogg.npy
Saved /kaggle/working/normoc/XC336743.ogg.npy
Saved /kaggle/working/normoc/XC416895.ogg.npy
Saved /kaggle/working/normoc/XC436355.ogg.npy
Saved /kaggle/working/normoc/XC629330.ogg.npy
Saved /kaggle/working/normoc/XC320716.ogg.npy
Saved /kaggle/working/norpin/XC637035.ogg.npy
Saved /kaggle/working/norpin/XC551679.ogg.npy
Saved /kaggle/working/norpin/XC653478.ogg.npy
Saved /kaggle/working/norpin/XC611







 71%|███████   | 10572/14852 [25:49<07:13,  9.86it/s]





 71%|███████   | 10576/14852 [25:49<06:26, 11.06it/s]





 71%|███████   | 10580/14852 [25:49<05:32, 12.87it/s]





 71%|███████▏  | 10584/14852 [25:50<05:03, 14.06it/s]

OSError: 936096 requested and 212864 written







 71%|███████▏  | 10584/14852 [26:08<05:03, 14.06it/s]

In [ ]:
# Load the dataset
file_path = '/kaggle/working/extra_data.csv'
df = pd.read_csv(file_path)

# 'Order' is the main target for stratified splitting at the top level
target_column = 'ORDER1'

# First split: Train (70%) and Temp (30%)
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df[target_column], random_state=42)
# Second split: Temp (30%) into Validation (15%) and Test (15%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df[target_column], random_state=42)


# Print the sizes to verify the splits
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

In [ ]:
!pip install tensorflow==2.12.0
!pip install efficientnet==1.1.1
!pip install librosa==0.9.2

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from efficientnet.tfkeras import EfficientNetB0
import librosa
import librosa.display
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
mel_spectrogram_dir = '/kaggle/working/'

features = []
labels = []
file_names = extra_data['filename'].tolist()
order_names = extra_data['ORDER1'].tolist()

for file_name, order_names in zip(file_names, order_names):
    # Construct the path to the Mel spectrogram image
    mel_spectrogram_path = os.path.join(mel_spectrogram_dir, file_name[:-4] + '.npy')
    
    if os.path.exists(mel_spectrogram_path):
        # Load the Mel spectrogram image
        img = load_img(mel_spectrogram_path, target_size=(128, 128))
        # Convert image to array
        img_array = img_to_array(img)
        # Append to features
        features.append(img_array)
        labels.append(order_names)

In [ ]:
features = np.array(features)
labels = np.array(labels)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels)

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3)) 
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(len(le.classes_), activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(features, labels, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
from sklearn.metrics import f1_score

# Predict on the test set
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"F1 Score: {f1}")